# Pruebas del Modelo UNet3 para Predicción de Heatmaps

Este notebook permite cargar un modelo entrenado y realizar pruebas adicionales, visualizando los resultados y calculando métricas.

In [ ]:
import os
import sys
import torch
import h5py
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime

# Agregar el directorio src al path para poder importar los módulos
sys.path.append('..')
from src.models.unet3 import UNet3
from src.data.dataset import HeatmapDataset
from src.metrics.metrics import calculate_metrics
from config.config import MODEL_CONFIG, DATA_CONFIG, METRICS_CONFIG

## 1. Cargar Modelo Entrenado

Primero cargamos el modelo entrenado desde los checkpoints guardados.

In [ ]:
def load_model(checkpoint_path):
    """Carga el modelo desde un checkpoint"""
    model = UNet3(
        n_channels=MODEL_CONFIG['input_frames'],
        n_classes=MODEL_CONFIG['output_frames']
    )
    
    # Determinar dispositivo
    device = "mps" if torch.backends.mps.is_available() else "cpu"
    
    # Cargar checkpoint
    checkpoint = torch.load(checkpoint_path, map_location=device)
    model.load_state_dict(checkpoint['state_dict'])
    model = model.to(device)
    model.eval()
    
    print(f"Modelo cargado desde: {checkpoint_path}")
    print(f"Epoch: {checkpoint['epoch']}")
    print(f"Validation Loss: {checkpoint['val_loss']:.4f}")
    
    return model

# Ruta al mejor modelo (ajusta según tu estructura)
checkpoint_path = "../saved_models/best_model.ckpt"
model = load_model(checkpoint_path)

## 2. Cargar Datos de Test

Cargamos el conjunto de datos de test para realizar las pruebas.

In [ ]:
# Cargar el dataset completo
test_dataset = HeatmapDataset(
    file_path=DATA_CONFIG['data_path'],
    input_frames=MODEL_CONFIG['input_frames'],
    output_frames=MODEL_CONFIG['output_frames'],
    subset='test'
)

print(f"Dataset de test cargado con {len(test_dataset)} muestras")

## 3. Realizar Predicciones y Visualizar Resultados

Seleccionamos algunas muestras aleatorias y visualizamos las predicciones.

In [ ]:
def visualize_prediction(model, dataset, idx):
    """Visualiza la predicción para una muestra específica"""
    inputs, target, timestamps = dataset[idx]
    
    # Preparar input para el modelo
    inputs_batch = inputs.unsqueeze(0).to(model.device)
    
    # Realizar predicción
    with torch.no_grad():
        prediction = model(inputs_batch)
    
    # Mover a CPU y convertir a numpy
    prediction = prediction.cpu().squeeze(0)
    
    # Desnormalizar si es necesario
    inputs = inputs * 100.0
    target = target * 100.0
    prediction = prediction * 100.0
    
    # Configurar visualización
    fig, axes = plt.subplots(1, 3, figsize=(18, 6))
    
    # Último frame de entrada
    im1 = axes[0].imshow(inputs[-1].numpy(), cmap='RdBu_r', vmin=0, vmax=100)
    axes[0].set_title(f'Input frame\n{timestamps[-1]}')
    axes[0].axis('off')
    
    # Predicción
    im2 = axes[1].imshow(prediction[0].numpy(), cmap='RdBu_r', vmin=0, vmax=100)
    axes[1].set_title(f'Prediction\n{timestamps[-1]}')
    axes[1].axis('off')
    
    # Target
    im3 = axes[2].imshow(target[0].numpy(), cmap='RdBu_r', vmin=0, vmax=100)
    axes[2].set_title(f'Target\n{timestamps[-1]}')
    axes[2].axis('off')
    
    # Barra de color común
    plt.colorbar(im2, ax=axes.ravel().tolist(), orientation='horizontal',
                pad=0.01, fraction=0.05, label='Intensidad')
    
    plt.tight_layout()
    plt.show()
    
    # Calcular métricas
    metrics = calculate_metrics(
        prediction[0],
        target[0],
        threshold=METRICS_CONFIG['threshold']
    )
    
    print("\nMétricas para esta predicción:")
    for metric, value in metrics.items():
        value_float = value.item() if isinstance(value, torch.Tensor) else value
        print(f"{metric.upper()}: {value_float:.4f}")
    
    return prediction, metrics

# Visualizar algunas predicciones aleatorias
n_samples = 3
random_indices = np.random.choice(len(test_dataset), n_samples)

for idx in random_indices:
    print(f"\nMuestra {idx}:")
    prediction, metrics = visualize_prediction(model, test_dataset, idx)

## 4. Evaluar Métricas en Todo el Conjunto de Test

Calculamos las métricas en todo el conjunto de test para tener una evaluación más completa.

In [ ]:
def evaluate_full_test_set(model, dataset):
    """Evalúa el modelo en todo el conjunto de test"""
    all_metrics = []
    
    for idx in range(len(dataset)):
        inputs, target, _ = dataset[idx]
        inputs_batch = inputs.unsqueeze(0).to(model.device)
        
        with torch.no_grad():
            prediction = model(inputs_batch)
        
        prediction = prediction.cpu().squeeze(0)
        metrics = calculate_metrics(
            prediction[0],
            target[0],
            threshold=METRICS_CONFIG['threshold']
        )
        all_metrics.append(metrics)
        
        if (idx + 1) % 10 == 0:
            print(f"Procesadas {idx + 1}/{len(dataset)} muestras...")
    
    # Calcular promedios
    avg_metrics = {}
    for metric in all_metrics[0].keys():
        values = [m[metric].item() if isinstance(m[metric], torch.Tensor) else m[metric] for m in all_metrics]
        avg_metrics[metric] = {
            'mean': np.mean(values),
            'std': np.std(values)
        }
    
    return avg_metrics

print("Evaluando todo el conjunto de test...")
test_metrics = evaluate_full_test_set(model, test_dataset)

print("\nResultados finales en test:")
for metric, stats in test_metrics.items():
    print(f"{metric.upper()}:")
    print(f"  Media: {stats['mean']:.4f}")
    print(f"  Desv. Est.: {stats['std']:.4f}")

## 5. Guardar Resultados

Guardamos los resultados de la evaluación para referencia futura.

In [ ]:
# Guardar resultados
results_dir = "../logs/test_results"
os.makedirs(results_dir, exist_ok=True)

timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
results_file = os.path.join(results_dir, f"test_results_{timestamp}.txt")

with open(results_file, 'w') as f:
    f.write("Resultados de Evaluación en Test\n")
    f.write("===============================\n\n")
    f.write(f"Modelo: {checkpoint_path}\n")
    f.write(f"Fecha: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n\n")
    
    f.write("Métricas:\n")
    for metric, stats in test_metrics.items():
        f.write(f"\n{metric.upper()}:\n")
        f.write(f"  Media: {stats['mean']:.4f}\n")
        f.write(f"  Desv. Est.: {stats['std']:.4f}\n")

print(f"\nResultados guardados en: {results_file}")